In [2]:
#import csv
import json
import requests
import pandas as pd
from tqdm import tqdm  # progress_apply
import postgis as pg
from pymeos import *
import pickle
# from tqdm.notebook import tqdm
import asyncio
import aiohttp
import os
from shapely.geometry import Point


# -----------------------
# CONFIGURATION
# -----------------------
SERVER_URL = "http://localhost:8080"
COLLECTION_ID = "ships"           # Change to your collection name


LIMIT_ROWS = None
BATCH_SIZE = 50


### based on PYMEOS example : https://github.com/MobilityDB/PyMEOS-Examples/blob/main/PyMEOS_Examples/AIS.ipynb
### -----------------------
### HELPER FUNCTIONS
### -----------------------

### rework the data using pandas and pymeos to get temporal geometry (trip)

In [3]:
tqdm.pandas()

pymeos_initialize()
# %%time
# get data, cleanup remove duplicates, wrong lat,long ///

ais = pd.read_csv(
    "./data/aisdk-2024-03-01.zip",
    usecols=["# Timestamp", "MMSI", "Latitude", "Longitude", "SOG"],
    nrows=100
)
ais.columns = ["t", "mmsi", "lat", "lon", "sog"]
# data preparation:  remove rows with no timestamps
ais = ais[ais["t"] != 0]
ais["t"] = pd.to_datetime(ais["t"], format='%d/%m/%Y %H:%M:%S')
ais = ais[ais["mmsi"] != 0]
ais = ais.drop_duplicates(["t", "mmsi"])
ais = ais[(ais["lat"] >= 40.18) & (ais["lat"] <= 84.17)]
ais = ais[(ais["lon"] >= -16.1) & (ais["lon"] <= 32.88)]
ais = ais[(ais["sog"] >= 0) & (ais["sog"] <= 1022)]
ais.dropna()
ais.head()

,t,mmsi,lat,lon,sog
0,2024-03-01,219000873,56.990910,10.304543,0.0
2,2024-03-01,219016683,56.800165,9.024933,0.0
3,2024-03-01,219000615,56.967093,9.224287,2.3
5,2024-03-01,219017664,56.974950,8.922530,0.0
6,2024-03-01,219002686,56.795143,8.863960,0.0


# Now, we will create the PyMEOS object representing
# the position and the SOG.

In [4]:

ais["inst"] = ais.progress_apply(
    lambda row: TGeomPointInst.from_base_time(Point(row["lon"], row["lat"]), row["t"]),
    axis=1
)

ais["sog_inst"] = ais.progress_apply(
    lambda row: TFloatInst(value=row["sog"], timestamp=row["t"]),
    axis=1
)

ais.drop(["lat", "lon"], axis=1, inplace=True)


trajectories = ais.groupby("mmsi").aggregate(
    {
        "inst": lambda x: TGeomPointSeq.from_instants(list(x), upper_inc=True),
        "sog_inst": lambda x: TFloatSeq.from_instants(list(x), upper_inc=True),
    }
).rename({"inst": "trajectory", "sog_inst": "sog"}, axis=1)


json_ready = []
for mmsi, row in trajectories.iterrows():
    print(type(row["trajectory"]))
    json_ready.append({
        "mmsi": int(mmsi),
        "trajectory": json.loads(row["trajectory"].as_mfjson()),
        "sog": json.loads(row["sog"].as_mfjson())
    })


with open("data/trajectories_mf1.json", "w") as f:
    json.dump(json_ready, f, indent=2)





100%|██████████| 54/54 [00:00<00:00, 2050.87it/s]


<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pymeos.main.tpoint.TGeomPointSeq'>
<class 'pym

# Assembling Trips (MEOS Example)
# Now, we will create the trajectory (TGeogPointSeq) and
# the SOG evolution (TFloatSeq) for every ship (identified by the mmsi)
# using the instants we have created.
# ::::::::::::::::::::::::::::stopped here , time limit

In [5]:
pymeos_finalize() 
# trajectories = (
#     ais.groupby("mmsi")
#     .aggregate(
#         {
#             "instant": lambda x: TGeogPointSeq.from_instants(x, upper_inc=True),
#             "sog": lambda x: TFloatSeq.from_instants(x, upper_inc=True),
#         }
#     )
#     .rename({"instant": "trajectory"}, axis=1)
# )
# trajectories["distance"] = trajectories["trajectory"].apply(
#     lambda t: t.length())

# trajectories.head()


In [6]:


# # Convert PyMEOS objects to MFJSON
# trajectories_json = trajectories.reset_index()  # mmsi becomes a column
# trajectories_json["trajectory"] = trajectories_json["trajectory"].apply(
#     lambda t: json.loads(t.as_mfjson())
# )
# trajectories_json["sog"] = trajectories_json["sog"].apply(
#     lambda t: json.loads(t.as_mfjson())
# )
# # 

# # Save to file
# os.makedirs("data", exist_ok=True)
# with open("data/trajectories_mf.json", "w") as f:
#     json.dump(trajectories_json.to_dict(orient="records"), f, indent=2)
# pymeos_finalize() 


# ::::::::::::::::::::::::::::::stopped here , time limit
# Storing in MobilityDB: so they use PYMEOS but here use server.py- susceptible d'etre modifié!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# https://stackoverflow.com/questions/51699817/python-async-post-requests


In [7]:

# def clean_up_resources():
 
#     try:
#         pymeos_finalize() 
#         print("MEOS resources cleaned up successfully.")
#     except Exception as e:
#         print(f"Error during resource cleanup: {str(e)}")

# server_url = 'http://localhost:8080/collections/{collectionId}/items'

# collection_id = 'ships'


# for index, row in trajectories.iterrows():
#     try:
    
#         post_data = {
#             "type":"Feature",
#             "id": row.name, 
#             "temporalGeometry":json.loads(row["trajectory"].as_mfjson())  
#         }

        
#         json_data = json.dumps(post_data)
# # post 
#         response = requests.post(
#             server_url.format(collectionId=collection_id), 
#             data=json_data, 
#             headers={'Content-Type': 'application/geo+json'}
#         )

     
#         if response.status_code == 200:
#             print(f"Successfully posted feature {row.name}")
#         else:
#             print(f"Failed to post feature {row.name}: {response.status_code} - {response.text}")
    
#     except requests.exceptions.RequestException as e:

#         print(f"Request error for feature {row.name}: {str(e)}")
    
#     except Exception as e:

#         print(f"Unexpected error for feature {row.name}: {str(e)}")

# clean_up_resources()
